In [1]:
import pandas as pd

print("Hello")
df = pd.read_csv("./data.csv", index_col="index")
print(df.head())


Hello
              host       time method                           url  response  \
index                                                                          
0      ***.novo.dk  805465029    GET                     /ksc.html       200   
1      ***.novo.dk  805465031    GET    /images/ksclogo-medium.gif       200   
2      ***.novo.dk  805465051    GET  /images/MOSAIC-logosmall.gif       200   
3      ***.novo.dk  805465053    GET     /images/USA-logosmall.gif       200   
4      ***.novo.dk  805465054    GET    /images/NASA-logosmall.gif       200   

       bytes  
index         
0       7067  
1       5866  
2        363  
3        234  
4        786  


In [2]:
# removing all not needed GET requests
remove_media = df[
    df.url.str.contains(".html", case=False) & df.url.str.contains(".htm", case=False)
    | df.url.str.endswith("/")
]


In [5]:
grouped_by_host = remove_media.groupby(by=["host"])
print(len(grouped_by_host))
# more_than_one_site = grouped_by_host.filter(lambda x: len(x) == 1)
more_than_one_site = grouped_by_host.filter(lambda x: len(x) > 1)


33805


In [11]:
print(len(more_than_one_site.groupby(by=["host"]).groups))
more_than_one_site.groupby(by=["host"]).head()


12149


,host,time,method,url,response,bytes
index,,,,,,
29,001.msy4.communique.net,809765747,GET,/software/winvn/winvn.html,200,9630
126,01.ts01.zircon.net.au,808994766,GET,/facts/faq04.html,200,27063
130,02-17-05.comsvc.calpoly.edu,809622206,GET,/shuttle/missions/sts-65/mission-sts-65.html,200,131166
144,023.msy4.communique.net,809316635,GET,/shuttle/missions/sts-64/mission-sts-64.html,200,58811
228,033.msy4.communique.net,808808644,GET,/shuttle/missions/sts-66/mission-sts-66.html,200,90112
...,...,...,...,...,...,...
703565,204.250.31.223,808541537,GET,/software/winvn/winvn.html,200,9866
703575,204.250.34.206,808286752,GET,/images/,200,17688
703642,204.250.59.114,808523201,GET,/shuttle/missions/sts-69/mission-sts-69.html,200,11836
